First recall what first class functions and closures are. See notebooks if needed. Here's a quick example.

In [95]:
from functools import wraps

In [1]:
def OuterFunction():
    message = 'Hi'
    def InnerFunction():
        print(message)
    return InnerFunction()


message is a free variable. 

In [2]:
OuterFunction()

Hi


In [3]:
def OuterFunction():
    message = 'Hi'
    def InnerFunction():
        print(message)
    return InnerFunction

If we take away the parentheses on the return statement, it returns the function which is waiting to be executed. I think once it's executed, that is the process of 'closure.'

In [4]:
myFunc=OuterFunction()

In [5]:
myFunc()

Hi


In [6]:
def OuterFunction(msg):
    message = msg
    def InnerFunction():
        print(message)
    return InnerFunction

In [7]:
hiFunc=OuterFunction('Hi')
byeFunc=OuterFunction('Bye')

In [8]:
hiFunc()

Hi


In [10]:
def OuterFunction(msg):
    def InnerFunction():
        print(msg)
    return InnerFunction

A decorator is a fucntion that takes ANOTHER function as an argument, adds some extra features, and then returns another function. ALl of this is done without altering the original function. So it's kind of a temporary add-on. 

In [12]:
def DecoratorFunction(message):
    def WrapperFunction():
        print(message)
    return WrapperFunction

Now what if instead of a string, our decorator fucntion accepts a different function?

In [13]:
def DecoratorFunction(origFunc):
    def WrapperFunction():
        origFunc()
    return WrapperFunction

In [15]:
def Display():
    print('display function ran')
    

In [26]:
decoratedDisplay = DecoratorFunction(Display)

Here, decoratedDisplay variable is set to the DecoratorFunction function, passed the Display function. The WrapperFunction then calls the passed function (Display) when executed.

In [20]:
decoratedDisplay()

display function ran


In [76]:
def DecoratorFunction(origFunc):
    def WrapperFunction(*args, **kwargs):
        print ('Wrapper executed this before {}'.format(origFunc.__name__))
        return origFunc(*args, **kwargs)
    return WrapperFunction

In [37]:
decoratedDisplay()

Wrapper executed this before Display
display function ran


But what about the @ sign??

In [80]:
@DecoratorFunction
def Display():
    print('display function ran')

In [81]:
Display()

Wrapper executed this before Display
display function ran


So the @ sign is saying that Display function will be passed as an argument to the DecoratorFunction every time we invoke the Display function? So that is identical to the following:

In [54]:
Display = DecoratorFunction(Display)

Another example

In [62]:
def DisplayInfo(name,age):
    print('DisplayInfo ran with arguments ({}, {})'.format(name,age))

In [63]:
DisplayInfo('Jon', 31)

DisplayInfo ran with arguments (Jon, 31)


Now what if we want to decorate this?

In [78]:
@DecoratorFunction
def DisplayInfo(name,age):
    print('DisplayInfo ran with arguments ({}, {})'.format(name,age))

In [79]:
DisplayInfo('Jon', 31)

Wrapper executed this before DisplayInfo
DisplayInfo ran with arguments (Jon, 31)


It does not work (or it did not work originally)! We need our wrapper to accept a flexible number of arguments. This is done using *args and **kwargs.

Classes can be used a decorators instead of functions.

In [82]:
class DecoratorClass(object):
    def __init__(self, origFunc):
        self.origFunc=origFunc
        
    def __call__(self, *args, **kwargs):
        print ('Call method executed this before {}'.format(self.origFunc.__name__))
        return self.origFunc(*args, **kwargs)

In [83]:
@DecoratorClass
def DisplayInfo(name,age):
    print('DisplayInfo ran with arguments ({}, {})'.format(name,age))

In [84]:
DisplayInfo('Jon', 31)

Call method executed this before DisplayInfo
DisplayInfo ran with arguments (Jon, 31)


That was kind of confusing. Might need to come back to it a second time.

Back to method decorators..

In [85]:
@DecoratorFunction
def DisplayInfo(name,age):
    print('DisplayInfo ran with arguments ({}, {})'.format(name,age))

In [86]:
DisplayInfo('Jon', 31)

Wrapper executed this before DisplayInfo
DisplayInfo ran with arguments (Jon, 31)


So what are some practicaL examples of decorators?

In [89]:
def MyLogger(origFunc):
    import logging
    logging.basicConfig(filename='{}.log'.format(origFunc.__name__),level=logging.INFO)
    @wraps(origFunc)
    def Wrapper(*args,**kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args,kwargs))
        return origFunc(*args, **kwargs)
    return Wrapper
            

In [90]:
@MyLogger
def DisplayInfo(name,age):
    print('DisplayInfo ran with arguments ({}, {})'.format(name,age))

In [91]:
DisplayInfo('Jon', 31)

DisplayInfo ran with arguments (Jon, 31)


Now this Logging decorator can be added whenever I want to log a function.

In [92]:
def MyTimer(origFunc):
    import time
    @wraps(origFunc)
    def Wrapper(*args,**kwargs):
        t1 = time.time()
        result = origFunc(*args, **kwargs)
        t2 = time.time()-t1
        print('{} ran in: {} sec'.format(origFunc.__name__, t2))
        return result
    return Wrapper


In [93]:
import time
@MyTimer
def DisplayInfo(name,age):
    time.sleep(1)
    print('DisplayInfo ran with arguments ({}, {})'.format(name,age))

In [94]:
DisplayInfo('Jon', 31)

DisplayInfo ran with arguments (Jon, 31)
DisplayInfo ran in: 1.000983476638794 sec


We can also chain decorators together. But to do this we need to add @wraps(origFunc) before EACH wrapper function.

In [96]:
import time
@MyLogger
@MyTimer
def DisplayInfo(name,age):
    time.sleep(1)
    print('DisplayInfo ran with arguments ({}, {})'.format(name,age))

In [97]:
DisplayInfo('Jon', 31)

DisplayInfo ran with arguments (Jon, 31)
DisplayInfo ran in: 1.0004572868347168 sec
